## GAHN Alioune
## DIALLO Aicha

Pour évaluer, la commande ``java -jar rouge2-1.2.1.jar `` marche trés bien sur le terminal en se placant dans la directory correspondante

##### Inspectons le dossier topics

In [1]:
def rdfiles(chemin):
    text_file = open("/home/sid2018-1/Bureau/Critiques/projects/test-summarization/topics/"+ chemin, "r", encoding = 'ISO 8859-1')
    lines1 = text_file.readlines()
    text_file.close()
    return(lines1)
    

**Lecture de chaque document**

In [2]:
f1 = rdfiles("battery-life_netbook_1005ha.txt.data")
f2 = rdfiles("price_amazon_kindle.txt.data")
f3 = rdfiles("room_holiday_inn_london.txt.data")
f4 = rdfiles("sound_ipod_nano_8gb.txt.data")
f5 = rdfiles("speed_windows7.txt.data")

**Nettoyage des documents**

In [3]:
def sentense2cleanTokens(sent):
    sent = "".join([x if (x.isalpha() or x.isnumeric() ) else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

In [4]:
f1 = [sentense2cleanTokens(x) for x in f1]
f2 = [sentense2cleanTokens(x) for x in f2]
f3 = [sentense2cleanTokens(x) for x in f3]
f4 = [sentense2cleanTokens(x) for x in f4]
f5 = [sentense2cleanTokens(x) for x in f5]

In [5]:
f1

['6GHz 533FSB cpu glossy display 3 Cell 23Wh Li ion Battery and a 1',
 'Not to mention that as of now Asus will not sell you a spare 3 or 6 cell Li Ion battery',
 'It also features a N270 cpu 6 cell 48Wh Li ion Battery 8',
 '3MP webcam 6 Cell 63Wh Li ion Battery with a whopping 10',
 'Realistic battery numbers are between 8',
 'of battery life if you re using wifi doing email word processing YouTube web surfing',
 'The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC s to boost performance or provide better battery life depending on their immediate requirements',
 'A few other things I d like to point out is that you must push the micro sized right angle end of the ac adapter until it snaps in place or the battery may not charge',
 'In addition the ac adapter does get quite hot while it is actively charging the battery which Asus clearly mentions in the user manual so one should not be alarmed over the heat output',
 '5 Hour Battery Glossy Display Bluetoo

### Question 3)  Faisons des statistics sur les documents

Le nombre de lignes de chaque document

In [6]:
nblines = [len(x) for x in [f1,f2,f3,f4,f5]]

Le nombre de mots de chaque documents 

In [7]:
nbwords = []
for k in [f1,f2,f3,f4,f5]:
    s= 0
    for j in k:
        s = s + len(j.split())
    nbwords.append(s)

In [8]:
nbwords

[6341, 1980, 12176, 1518, 2138]

Calculons les mots les plus fréquents pour chaque document

In [9]:
mostfrequentwords = []


In [10]:
corpus = []
for k in [f1,f2,f3,f4,f5]:
    s =  ''
    for j in k: 
        s = s+ ' '+ j
    corpus.append(s)

In [11]:
len(corpus)

5

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words= 'english')
X = vectorizer.fit_transform(corpus)
X= (X.toarray())  


In [13]:
for j in (X):
    mostfrequentwords.append(vectorizer.get_feature_names()[list(j).index(max(j))] )

In [14]:
mostfrequentwords

['battery', 'price', 'room', 'sound', 'faster']

Les mots les plus fréquents de chaque document sont: 
doc 1: 'battery' 
doc 2: 'price'
doc 3: 'room'
doc 4: 'sound'
doc 5: 'faster'

#### Question 4) Calculons les similarités entre les phrases du document 1

On calcule les similarités avec une premiere méthode consistant à considerer les représentations issues du tf-idf

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tf1 = tfidf.fit_transform(f1)
tf1= (tf1.toarray())  


In [69]:
tf1.shape

(333, 1256)

In [71]:
mat1 = np.zeros((1256,1256))

In [73]:
from scipy import spatial
for i,k in enumerate(tf1):
    for o,j in enumerate(tf1):
        if(i > o):
            mat1[i,o] =  1 - spatial.distance.cosine(k, j)
            mat1[o,i] = 1 - spatial.distance.cosine(k, j)

In [74]:
mat1

array([[0.        , 0.23749496, 0.41126425, ..., 0.        , 0.        ,
        0.        ],
       [0.23749496, 0.        , 0.25739253, ..., 0.        , 0.        ,
        0.        ],
       [0.41126425, 0.25739253, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Question 5) Calcul des Résumés de Chaque document 

Ici, nous calculons les résumés pour chaque document en renvoyant les 5 phrases les mieux scorées. 
Nous calculons ici le score de chaque phrase en prenant la somme des idf des mots qu'elle contient. 
L'idée est de renvoyer les 5 phrases avec les mots qui ont un plus grand IDF. 

In [81]:
# Fonction permettant de calculer la score de chaque phrase en prenant la somme des idf des mots qu'elle contient
def scoreph ( f):
    li = []
    for k  in f :
        l = 0 
        for j in k.split():
            s = [1 if j in x else 0 for x in f ]
            l = l+ (len(f)/sum(s))
            
        li.append(l)   
    return (li)

In [82]:
len(scoreph(f1)),(len(f1))

(333, 333)

In [93]:
# Resumé du document 1
resume = []
for i in range(5):
    element = f1[scoreph(f1).index(max(scoreph(f1)))]
    resume.append(element) 
    f1.remove(element)

In [96]:
resume

['Her computer is a top of the line notebook that never leaves her desk because of bulkiness and pathetic battery life so her only backdrop is the block wall of her dorm XD but with this eeepc she never sees me in the same place twice and almost all of the time im outside far from buildings which really says something about the wireless card',
 'My only other gripe is the speakers are very weak for media use but there are pocket able battery powered speaker options I am using the Altec Lansing Orbit Altec Lansing iM 237 Orbit Ultraportable Speaker for MP3 Players which is great',
 'Now add in the N standard wireless free netbook sleeve and the usual laptop microphone webcam suite and together with its aforementioned compactness and long battery life it becomes possible to do productive work on the Asus Eee HA1005 maintaining web sites writing long articles coding e networking etc almost anywhere',
 'The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC s t

In [97]:
resume2 = []
for i in range(5):
    element = f2[scoreph(f2).index(max(scoreph(f2)))]
    resume2.append(element) 
    f2.remove(element)
resume3 = []
for i in range(5):
    element = f3[scoreph(f3).index(max(scoreph(f3)))]
    resume3.append(element) 
    f3.remove(element)
resume4 = []
for i in range(5):
    element = f4[scoreph(f4).index(max(scoreph(f4)))]
    resume4.append(element) 
    f4.remove(element)
resume5 = []
for i in range(5):
    element = f5[scoreph(f5).index(max(scoreph(f5)))]
    resume5.append(element) 
    f5.remove(element)


In [98]:
resume2

['Before purchasing I was obsessed with the reviews and predictions I found online and reading about some of the critiques such as the thick border the lack of touchscreen lack of battery SD slot lack of a back light awkward difficult keyboard layout minimally faster page flipping and the super high price',
 'However I will say the selection of inexpensive books has been worth the purchase price of this item especially for those that enjoy reading the classics or nationwide news it has been a joy to purchase entire collections for a fraction of the cost of purchasing new or even used books',
 'To have this continuous high speed wireless connectivity without the need for a physical computer to upload download onto is truly amazing and makes the price of the Kindle truly irrelevant',
 'There are many other ways such as mp3 players portable DVD players not to mention the radio to listen to while driving that are almost fully paid for by the price for just one audio book',
 'I want you to 

In [99]:
resume3

['firm comfortable beds windows that open adequate mirrors clothes drying line across bathtub sinks and showers drained properly stopper in sink sealed which was good because we did our own laundry night shades overlapped the window and each other enough to keep light out mini bar refrigerator ran quietly free newspapers daily in elevator waiting areas in room safe American sized wash cloths all lights worked enough electrical outlets enough hangers central air conditioning which worked and ran quietly adequate elevators no toilet problems shower curtains sealed water from leaking onto the floor anti slip surface on bathtub shower exhaust system in bathroom removed condensation from shower always enough hot water in shower',
 'good size considered it is in london comfy double bed clean in room tea coffee choc making facilities hairdryer under desk old tv toilet facilities was disgusting long curly black hair in bathtub big black pubic hair tangled in hand towel disgusting yellow scum s

### Question 6) **enregistrons maintenant les résultats pour pouvoir les évaluer**

In [100]:
with open('/home/sid2018-1/Bureau/Critiques/projects/test-summarization/system/battery-life_netbook_1005ha.txt.sys.head5', 'w') as f:
    for item in resume:
        f.write("%s\n" % item)

In [101]:
with open('/home/sid2018-1/Bureau/Critiques/projects/test-summarization/system/price_amazon_kindle.txt.sys.head5', 'w') as f:
    for item in resume2:
        f.write("%s\n" % item)

In [102]:
with open('/home/sid2018-1/Bureau/Critiques/projects/test-summarization/system/room_holiday_inn_london.txt.sys.head5', 'w') as f:
    for item in resume3:
        f.write("%s\n" % item)

In [103]:
with open('/home/sid2018-1/Bureau/Critiques/projects/test-summarization/system/sound_ipod_nano_8gb.txt.sys.head5', 'w') as f:
    for item in resume4:
        f.write("%s\n" % item)

In [104]:
with open('/home/sid2018-1/Bureau/Critiques/projects/test-summarization/system/speed_windows7.txt.sys.head5', 'w') as f:
    for item in resume5:
        f.write("%s\n" % item)

#### Question 7) Evaluons les résultats

``` 
========Results Summary=======

ROUGE-L+StopWordRemoval	PRICE	AMAZON	Average_R:0,66667	Average_P:0,09195	Average_F:0,16162	Num Reference Summaries:1


ROUGE-1+StopWordRemoval	PRICE	AMAZON	Average_R:0,69231	Average_P:0,08571	Average_F:0,15254	Num Reference Summaries:1


ROUGE-2+StopWordRemoval	PRICE	AMAZON	Average_R:0,00000	Average_P:0,00000	Average_F:0,00000	Num Reference Summaries:1


ROUGE-SU4+StopWordRemoval	PRICE	AMAZON	Average_R:0,24444	Average_P:0,02316	Average_F:0,04231	Num Reference Summaries:1


ROUGE-L+StopWordRemoval	SOUND	IPOD	Average_R:0,28571	Average_P:0,02941	Average_F:0,05333	Num Reference Summaries:1


ROUGE-1+StopWordRemoval	SOUND	IPOD	Average_R:0,42857	Average_P:0,03448	Average_F:0,06383	Num Reference Summaries:1


ROUGE-2+StopWordRemoval	SOUND	IPOD	Average_R:0,00000	Average_P:0,00000	Average_F:0,00000	Num Reference Summaries:1


ROUGE-SU4+StopWordRemoval	SOUND	IPOD	Average_R:0,18750	Average_P:0,00779	Average_F:0,01496	Num Reference Summaries:1


ROUGE-L+StopWordRemoval	BATTERY-LIFE	NETBOOK	Average_R:0,50000	Average_P:0,03670	Average_F:0,06838	Num Reference Summaries:1


ROUGE-1+StopWordRemoval	BATTERY-LIFE	NETBOOK	Average_R:0,55556	Average_P:0,03846	Average_F:0,07194	Num Reference Summaries:1


ROUGE-2+StopWordRemoval	BATTERY-LIFE	NETBOOK	Average_R:0,14286	Average_P:0,00800	Average_F:0,01515	Num Reference Summaries:1


ROUGE-SU4+StopWordRemoval	BATTERY-LIFE	NETBOOK	Average_R:0,28000	Average_P:0,01167	Average_F:0,02240	Num Reference Summaries:1


ROUGE-L+StopWordRemoval	ROOM	HOLIDAY	Average_R:0,25000	Average_P:0,00935	Average_F:0,01802	Num Reference Summaries:1


ROUGE-1+StopWordRemoval	ROOM	HOLIDAY	Average_R:0,25000	Average_P:0,00719	Average_F:0,01399	Num Reference Summaries:1


ROUGE-2+StopWordRemoval	ROOM	HOLIDAY	Average_R:0,00000	Average_P:0,00000	Average_F:0,00000	Num Reference Summaries:1


ROUGE-SU4+StopWordRemoval	ROOM	HOLIDAY	Average_R:0,08696	Average_P:0,00149	Average_F:0,00293	Num Reference Summaries:1


ROUGE-L+StopWordRemoval	SPEED	WINDOWS7.TXT.SYS.HEAD5	Average_R:0,20000	Average_P:0,03093	Average_F:0,05357	Num Reference Summaries:1


ROUGE-1+StopWordRemoval	SPEED	WINDOWS7.TXT.SYS.HEAD5	Average_R:0,18750	Average_P:0,02256	Average_F:0,04027	Num Reference Summaries:1


ROUGE-2+StopWordRemoval	SPEED	WINDOWS7.TXT.SYS.HEAD5	Average_R:0,07692	Average_P:0,00781	Average_F:0,01418	Num Reference Summaries:1


ROUGE-SU4+StopWordRemoval	SPEED	WINDOWS7.TXT.SYS.HEAD5	Average_R:0,08000	Average_P:0,00650	Average_F:0,01203	Num Reference Summaries:1


======Results Summary End====== 
```

Aprés évaluation des résultats, nous n'améliorons pas les résultats car notre méthode renvoie les phrases les plus longues(celles qui ont un plus grand nombres de mots ont plus de chances d'avoir une plus grande somme des idf) et notre méthode ne prend pas en compte les dissimilarités entre les phrases afin de renvoyer un résumé qui est plus diversifié

#### question 8) Essayons une nouvelle méthode

Une méthode qui marcherait bien ici est de faire un Kmeans en 5 clusters pour chaque document. 
Ensuite pour chaque cluster il suffirait de choisir la phrase ayant la plus grande somme des IDF de ses mots. 
Cela permettrai d'avoir un résumé diversifié et riche. 
(La différence entre cette approche et notre méthode est qu'on ne fait pas l'étape de clustering qui permets d'obtenir 
le résumé diversifié)